# Unsupervised Learning

Overview

In the past two lessons, you have learned what Unsupervised Machine Learning is, what problems are suitable for a solution based on Unsupervised Machine Learning, how to apply Unsupervised Machine Learning, and you have practiced implementing the basic phases of a solution using Scikit-learn. Now is time to put all that conceptual and procedural knowledge to work by doing a larger project. Choose a problem domain that motivates you, and build a complete solution implementing all the phases you learned about in previous chapters. We provide some ideas of interesting problem domains in a dedicated section in this lesson, but we want you to be creative and adventurous, and explore other options as well. This lesson does not present any new material: everything you will need to complete this project was discussed on previous lessons.
External Interface Requirements

    Input requirement: capacity to read a dataset stored on disk.
    Output requirement: report on optimal number of clusters, centroid coordinates and quality metric.
    Output requirement: identifiers of classes corresponding to new instances classified by the model.

Functional Requirements

    The software must learn a clusterization a the dataset.
    The software must use the learned clusterization to classify new problem instances.
    The software must evaluate the quality of a clusterization.
    The software must be flexible to work with different preconfigured amount of clusters.
    The software must compare results using different numbers of clusters and determine which number of clusters is best.

Technical Requirements

    Use Python as programming language.
    Use Pandas for reading the dataset into a Pandas dataframe.
    Use Scikit-learn for training and testing the Machine Learning model.

Necessary Deliverables

    Python application that performs ETL, training, and testing.
    Report containing quality metrics, and explanation of the dataset, and the experimental procedure (range of the different number of clusters that were tested, how the range was traversed, etc.).
    Optional(Build a supervised model by attaching your labels(clusters) as Target)

Suggestions to Get Started

    Find an interesting dataset! Look in the Useful Resources section for sources of ideas.
    If you do not find a pre-existing dataset on the problem domain that you like, be creative: consider building the dataset yourself and donating the dataset to one of the public Machine Learning repositories.
    Break down the project into smaller tasks, for instance: importing the dataset, training, etc.
    Decide whether you will create a single Python application or several Python applications.

Potential Project Ideas

    Segment smartphone users according to phone usage and apps installed.
    Segment healthy person under 50 years of age according to their risk or propensity of suffering from Alzheimer's disease after 70 years of age.
    Classify differnent customers from an Ecomerce data.


In [ ]:
# starting libraries, additional ones will be added when needed

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly

%matplotlib inline
sns.set()

In [ ]:
# display options

pd.options.display.max_columns = None
pd.set_option('display.max_rows', 200)

### data import and cleaning

In [ ]:
df = pd.read_csv(r"C:\Users\Aciago\ih\Week7_project\data\terror\globalterrorismdb_0718dist.csv", encoding='latin-1')

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
percent_missing = pd.DataFrame((df.isnull().sum() * 100 / len(df)).sort_values(ascending=False))
percent_missing

In [ ]:
df1 = df.loc[:, df.isnull().mean() < .15]

In [ ]:
df1.shape

In [ ]:
df1.isna().sum()

In [ ]:
df1 = df1.drop_duplicates()

In [ ]:
df1['event_date']  = (pd.to_datetime(df1['iday'].astype(str) + '-' +
                                  df1['imonth'].astype(str) + '-' +
                                  df1['iyear'].astype(str), errors='coerce'))

# errors{‘ignore’, ‘raise’, ‘coerce’}, default ‘raise’
# coerce will return NaT

In [ ]:
df1 = df1.dropna(axis=0)
# df1 = df1.drop(['iyear','iday','imonth'], axis=1)

In [ ]:
df1.shape

In [ ]:
df1.isna().sum()

In [ ]:
df1.gname.value_counts()
#txt remove, country and region, targtype1_txt, 
# reshape date col into month=+1

In [ ]:
# merge 2 cols regarding victims into single one
df1['Victims']=df1['nkill']+df1['nwound']

In [ ]:
df1.drop(['nkill','nwound'],axis=1,inplace=True)

In [ ]:
#df1.drop(['eventid'],axis=1,inplace=True)
df1.reset_index(drop=True, inplace=True)

In [ ]:
# renaming cols to a more logical and readable format

df1.rename(columns={'extended':'Extended', 'country':'Country_id','country_txt':'Country_txt',
                    'region':'Region_id', 'region_txt':'Region_txt', 'provstate':'Provstate', 'city':'City',
                    'success':'Success', 'doubtterr':'PossibleNonTA','suicide':'Suicide', 'attacktype1':'AttackType_id',
                    'attacktype1_txt':'AttackType_txt', 'targtype1':'TargetType_id', 'target1':'Target',
                    'nkill':'Killed','nwound':'Wounded','summary':'Summary', 'Target_type':'TargetType_txt',
                    'targsubtype1':'TargetSubType_id', 'targsubtype1_txt':'TargetSubType_txt',
                    'natlty1':'TO_Nation_id','natlty1_txt':'TO_Nation_txt', 'weaptype1':'WeaponType_id',
                    'weapsubtype1':'WeaponSubType_id','weapsubtype1_txt':'WeaponSubType_txt',
                    'weaptype1':'WeaponType_id', 'targtype1_txt':'TargetType_txt', 'multiple':'MultipleActs',
                   'gname':'TO_name', 'weaptype1_txt':'WeaponType_txt','property':'PropertyDmg'},inplace=True)

In [ ]:
df1.to_csv('terror_clean.csv')

## raw data viz

In [ ]:
plt.subplots(figsize=(22,8))

sns.countplot('iyear', data=df1, palette='RdYlGn_r', orient='h') # orient arg ignored?

plt.title('Reported Acts of Terror per Year')
plt.xlabel('Year')
plt.ylabel('Number of reports')

plt.show();

In [ ]:
plt.subplots(figsize=(15,6))
sns.countplot('WeaponType_txt', data=df1, palette='RdYlGn', order=df1['WeaponType_txt'].value_counts().index)

plt.title('Weapons used in Terrorism')
plt.xlabel('Weapon type')
plt.ylabel('Number of attacks')

plt.show();

In [ ]:
plt.subplots(figsize=(15,6))
sns.countplot('Region_txt', data=df1, palette='RdYlGn', order=df1['Region_txt'].value_counts().index)

plt.title('Amount of Terrorism Acts per Region')
plt.xlabel('Region name')
plt.ylabel('Number of attacks')
plt.xticks(rotation=35)

plt.show();

In [ ]:
plt.figsize=(15,6)
df1.hist()
plt.show()

In [ ]:
# we will drop text columns that already have numerical representation in the dataset, cols w\o meaning and datetime

df2 = df1.drop(['eventid', 'iyear', 'imonth', 'iday', 'Country_txt', 'Region_txt',
                'Provstate', 'AttackType_txt', 'TargetType_txt', 'Target','TargetSubType_txt',
               'WeaponType_txt', 'TO_Nation_txt', 'WeaponSubType_txt', 'dbsource', 'event_date', 'City'], axis=1)

In [ ]:
#df2.dbsource.value_counts()
df2.TO_name.value_counts()[0:10]

In [ ]:
s = df2.TO_name.value_counts()

df2['Org_claim'] = np.where(df2['TO_name'].isin(s.index[s >= 1560]), df2['TO_name'], 'Other')

In [ ]:
df2 = pd.get_dummies(data=df2, columns=['Org_claim'], drop_first=True)
df2.drop('TO_name', axis=1, inplace=True)

In [ ]:
df2.shape

## Preprocessing

In [ ]:
# Rescaling method
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data=df2.values
x=data[:,0:8]
y=data[:,8]

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
rescaledX=scaler.fit_transform(x)
rescaledX

In [ ]:
# Standardize method
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler1=StandardScaler().fit(x)
rescaledX1=scaler1.transform(x)
rescaledX1

### model building

In [ ]:
# clustering - KMeans (sklearn.cluster import KMeans)

# silhoutte score
# from yellowbrick.cluster import KElbowVisualizer

#### k-Means

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans=KMeans(n_clusters=40)
df2_clusters=kmeans.fit(rescaledX)

In [ ]:
df2_clusters

In [ ]:
centroids=kmeans.cluster_centers_
labels=kmeans.labels_
centroids

In [ ]:
labels

In [ ]:
# any X and Y columns, hue by centroids

In [ ]:
df2.sample(3)

In [ ]:
sns.scatterplot(x=df2.Victims, y=df2.Country_id, hue=centroids);

In [ ]:
# sns.scatterplot(x=df2.weaptype1, y=df2.country, hue=labels);

In [ ]:
# sns.scatterplot(x=df2.weaptype1, y=df2.success, hue=labels);

In [ ]:
plt.scatter(centroids[:,0],centroids[:,1],marker='x',s=150,linewidths=5,zorder=10)

In [ ]:
# how to viz actual clusters ffs?

from yellowbrick.cluster import KElbowVisualizer
from sklearn import cluster

In [ ]:
df2.head()

In [ ]:
n_samples=1500
X,y=df2(['Country_id', 'TargetType_id'])

In [ ]:
model=cluster.KMeans()

In [ ]:
visualizer=KElbowVisualizer(model, k=(1,5))
visualizer.fit(X)
visualizer.poof()

## DBScan model

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
stscaler = StandardScaler().fit(df2)
data = stscaler.transform(df2)

In [ ]:
plt.figsize=(15,10)
plt.scatter(df2.TargetType_id, df2.Country_id)
plt.xlabel("Type of target")
plt.ylabel("Country")
plt.title("test viz")
plt.show()
#plt.savefig("results/wholesale.png", format = "PNG")

In [ ]:
print(type(stscaler))
print(type(data))

In [ ]:
print(df2.shape)
df2.head()

In [ ]:
clustering = DBSCAN(algorithm='auto', metric='euclidean', eps=10, min_samples=50, n_jobs=-1).fit(data)
clustering.labels_

In [ ]:
?sklearn.metrics.pairwise_distances

In [ ]:
dbsc = DBSCAN(eps = 3, min_samples = 10).fit(data)

In [ ]:
labels = dbsc.labels_
core_samples = np.zeros_like(labels, dtype = bool)
core_samples[dbsc.core_sample_indices_] = True

In [ ]:
unique_labels = np.unique(labels)
colors = plt.cm.Spectral(np.linspace(0,1, len(unique_labels)))

In [ ]:
for (label, color) in zip(unique_labels, colors):
    class_member_mask = (labels == label)
    xy = data[class_member_mask & core_samples]
    plt.plot(xy[:,0],xy[:,1], 'o', markerfacecolor = color, markersize = 10)
    
    xy2 = data[class_member_mask & ~core_samples]
    plt.plot(xy2[:,0],xy2[:,1], 'o', markerfacecolor = color, markersize = 5)
plt.title("DBSCAN on Wholsesale data")
plt.xlabel("Grocery (scaled)")
plt.ylabel("Milk (scaled)")
#plt.savefig("results/dbscan_wholesale.png", format = "PNG")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

plt.figure(figsize=(12, 12))

test_data = np.asarraydf2['']
n_samples = 1500
random_state = 42
X, y = df2(n_samples=n_samples, random_state=random_state)

# Incorrect number of clusters
y_pred = KMeans(n_clusters=10, random_state=random_state).fit_predict(X)

plt.subplot(221)
plt.scatter(X[:, 0], X[:, 1], c=y_pred)
plt.title("test")

In [ ]:


# Anisotropicly distributed data
transformation = [[0.60834549, -0.63667341], [-0.40887718, 0.85253229]]
X_aniso = np.dot(X, transformation)
y_pred = KMeans(n_clusters=3, random_state=random_state).fit_predict(X_aniso)

plt.subplot(222)
plt.scatter(X_aniso[:, 0], X_aniso[:, 1], c=y_pred)
plt.title("Anisotropicly Distributed Blobs")

# Different variance
X_varied, y_varied = make_blobs(n_samples=n_samples,
                                cluster_std=[1.0, 2.5, 0.5],
                                random_state=random_state)
y_pred = KMeans(n_clusters=3, random_state=random_state).fit_predict(X_varied)

plt.subplot(223)
plt.scatter(X_varied[:, 0], X_varied[:, 1], c=y_pred)
plt.title("Unequal Variance")

# Unevenly sized blobs
X_filtered = np.vstack((X[y == 0][:500], X[y == 1][:100], X[y == 2][:10]))
y_pred = KMeans(n_clusters=3,
                random_state=random_state).fit_predict(X_filtered)

plt.subplot(224)
plt.scatter(X_filtered[:, 0], X_filtered[:, 1], c=y_pred)
plt.title("Unevenly Sized Blobs")

plt.show()